# Dataframe from JSON


In [1]:
import pyspark
import pyspark.sql.functions as F
import os
import pandas as pd
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, MapType
from pyspark.sql.functions import udf

## The 'pyarrow' lib provides a considerable performance improvement. But, it doesn't support ArrayType


In [2]:
spark = SparkSession.builder \
.config("spark.sql.execution.arrow.pyspark.enabled", "true") \
.appName('test').master("spark://127.0.0.1:7077")\
.getOrCreate()

23/09/24 12:13:22 WARN Utils: Your hostname, eletricage resolves to a loopback address: 127.0.1.1; using 192.168.15.6 instead (on interface wlp4s0)
23/09/24 12:13:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/24 12:13:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


The JSON file test.js

```json
[
    {
        "name": "Andre",
        "id": 1,
        "doc_list":[{"docid":"DOC001", "name":"bla001.txt"}, {"docid":"DOC002", "name":"bla002.txt"}],
    },

    {
        "name": "Noé",
        "id": 1,
        "doc_list":[{"docid":"DOC003", "name":"bla003.txt"}, {"docid":"DOC004", "name":"bla004.txt"}],
    }
]


```

The easiest way to read a local file is import it using Pandas and convert it into a DataFrame object later.



## The problem

Besides to read the JSON file, off course, suppose that its desirable to extract the doc file names associated to the people's names. Note that for the each name there is a list of docs with 'docid' and 'name' belonged to the docs. How can we get a list of doc names for each person name in a new column called 'doc_names'?

In [4]:
# Reading file using Pandas
jdf = pd.read_json('test.js')
# Converting to Spark dataframe
sdf = spark.createDataFrame(jdf)
# Showing the result
sdf.show(truncate=False)

/home/andre/Projects/Spark/spark-standalone/apps/venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:329: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(StructType(List(StructField(docid,StringType,true),StructField(name,StringType,true))),true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)
/home/andre/Projects/Spark/spark-standalone/apps/venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+-----+---+------------------------------------------------------------------------------+
|name |id |doc_list                                                                      |
+-----+---+------------------------------------------------------------------------------+
|Andre|1  |[{name -> bla001.txt, docid -> DOC001}, {name -> bla002.txt, docid -> DOC002}]|
|Noé  |1  |[{name -> bla003.txt, docid -> DOC003}, {name -> bla004.txt, docid -> DOC004}]|
+-----+---+------------------------------------------------------------------------------+



User Defined Functions(UDF) is a way to parse information from a column. In this case, the docs inside the JSON file is available in a list of objects which is parsed by pySpark and convenient converted into Python data structure objects. In this case, a list of dictionaries which is eaiser to manipulate. Brilliant!

In [5]:
@udf
def extract_doc_udf(data_list):
    n = list()
    for li in data_list:
        
        n += [v for k,v in li.items() if k == 'name']

    return ', '.join(n)


In [7]:
# Running the UDF called 'extract_doc_udf' and storing into a new column called 'udf_res'
dfu = sdf.withColumn('doc_list', extract_doc_udf(F.col('doc_list')))
# Showing the result
dfu.show(truncate=False)

+-----+---+----------------------+
|name |id |doc_list              |
+-----+---+----------------------+
|Andre|1  |bla001.txt, bla002.txt|
|Noé  |1  |bla003.txt, bla004.txt|
+-----+---+----------------------+



Alternativelly, it's possible to use a simple Python function passing the dataframe row as a parameter. But, to do that is necessary to use RDD framework instead UDF and then convert it to DataFrame object later. This way is useful when you parse different fields in a row in iteractive way. But, note that the performance will drop considerably depending on data amount.

In [12]:
def extract_doc_rdd(row):
    d = row.asDict()
    n = list()
    if 'doc_list' in d:
        for li in d['doc_list']:
            n += [v for k,v in li.items() if k == 'name']

        d['doc_names'] = ', '.join(n)

    return Row(**d)
    


In [13]:
# Executing 'extract_doc_rdd' using map method from rdd object
rdd = sdf.rdd.map(extract_doc_rdd)
# Converting into a dataframe object
edf = rdd.toDF()
# Showing the result
edf.show(truncate=False)

+-----+---+------------------------------------------------------------------------------+----------------------+
|name |id |doc_list                                                                      |doc_names             |
+-----+---+------------------------------------------------------------------------------+----------------------+
|Andre|1  |[{name -> bla001.txt, docid -> DOC001}, {name -> bla002.txt, docid -> DOC002}]|bla001.txt, bla002.txt|
|Noé  |1  |[{name -> bla003.txt, docid -> DOC003}, {name -> bla004.txt, docid -> DOC004}]|bla003.txt, bla004.txt|
+-----+---+------------------------------------------------------------------------------+----------------------+

